In [ ]:
# This script installs all required libraries for data analysis, plotting, LLM workflows, and notebook imports.

# pandas: Data manipulation and analysis
# numpy: Numerical computations
# matplotlib: Data visualization
# yfinance: Downloading financial data from Yahoo Finance
# langchain: Building LLM-powered applications and chains
# import_ipynb: Importing Jupyter notebooks as Python modules
# scipy: Scientific computing (e.g., signal processing)
# statsmodels: Statistical modeling and time series analysis
# xgboost: Gradient boosting for machine learning
# scikit-learn: Machine learning utilities (train/test split, etc.)

%pip install pandas numpy matplotlib yfinance langchain import_ipynb scipy statsmodels xgboost scikit-learn

In [8]:
import os
import copy
# -------------------------------------------------------------------------
#  LangChain Imports
# -------------------------------------------------------------------------
from langchain.chains import SequentialChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI  # Replace with any LLM provider
from langchain.output_parsers import RegexParser
# -------------------------------------------------------------------------
# GOLD_ETF Imports
# -------------------------------------------------------------------------
from Jaison.GOLD_ETF_functions import *
from Tarun.Gold_Price_TS_Model import *


In [9]:
import statsmodels.api as sm
import numpy as np

# Load the pre-trained ARIMAX model
arimax_model = sm.load_pickle('Jaison/Model/Arimax/arimax_2025-06-19.pkl')
# --------------------------------------------------------------------
# Example sentiment score for testing
news_sentiment = 0  # Example sentiment score
# --------------------------------------------------------------------
# Predict gold price using the loaded ARIMAX model and existing news_sentiment
predicted_price, next_day_pct_change, next_day = arimax_gold_price_prediction(news_sentiment, arimax_model)

print(f"Predicted gold price for {next_day.strftime('%Y-%m-%d')}: {predicted_price}")
print(f"Next day percentage change: {next_day_pct_change:.2f}%")
# --------------------------------------------------------------------

Predicted gold price for 2025-06-20: 82.56724627377722
Next day percentage change: -0.24%


In [ ]:
## News Data Download and Preprocessing
## Import the libraries

In [ ]:
## News data sentiment analysis using LLM

In [ ]:
# Dummy block to combine price data and news data prediction output

# Assume price_pred and news_pred are prediction outputs from previous steps
# For demonstration, we'll create dummy values
price_pred = 1800  # Example: predicted gold price from price model
news_pred = 10     # Example: sentiment score or predicted impact from news model

# Combine predictions (simple weighted sum as a placeholder)
final_price_output = 0.7 * price_pred + 0.3 * news_pred

print(f"Final predicted gold price: {final_price_output}")